#Keras implementation
This is an implementation on Google Colab Notebook of the deep learning classification model constructed using stacked nonsymmetric deep autoencoder (NDAE) and Random Forest algorithm on KDD Cup'99 dataset. This model is proposed in the article "A Deep Learning Approach to Network Intrusion Detection" (https://ieeexplore.ieee.org/document/8264962)



In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


#Prepare data
The KDD Cup’99 dataset is loaded from .csv files, which contains 494,021 training records and 311,029 testing records.

This dataset needs pre-processing to be successfully utilised with stacked NDAE model.This is because our model operates using only numeric values but one record in the dataset has a mixture of numeric and symbolic values, so a data transformation was needed to convert them. In addition integer values also need normalisation as they were mixed with ﬂoating point values between 0 and 1, which would make learning difﬁcult.



Load data from .csv file

In [0]:
import pandas
data_train = pandas.read_csv('/content/drive/My Drive/NDAE_IEEE2018/data/kddcup.data_10_percent_train_5_class.csv')
data_test = pandas.read_csv('/content/drive/My Drive/NDAE_IEEE2018/data/kddcup.data_10_percent_test_5_class.csv')
x_train =data_train.iloc[:, :-1].values
y_train = data_train.iloc[:, 41].values
x_test =data_test.iloc[:, :-1].values
y_test = data_test.iloc[:, 41].values

Firstly, the symbolic features: Protocol type, Service, Flag are transformed to numeric type, then these are encoded by one-hot vectors.

In [0]:

import numpy as np
#Concate training and testing data to do the same preprocess steps
x = np.concatenate((x_train, x_test), axis = 0)

#Transform to numeric features
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Encode Protocol type feature (3 different values)
labelencoder_x_1 = LabelEncoder()
x[:, 1] = labelencoder_x_1.fit_transform(x[:, 1])
# Encode Service feature (67 different values)
labelencoder_x_2 = LabelEncoder()
x[:, 2] = labelencoder_x_2.fit_transform(x[:, 2])

# Encode Flag feature (11 different values)
labelencoder_x_3 = LabelEncoder()
x[:, 3] = labelencoder_x_3.fit_transform(x[:, 3])


# Encoded by one-hot vector
from sklearn.compose import ColumnTransformer
# transform the second feature to 3-dim one-hot vector
ct = ColumnTransformer([("ProtocolType", OneHotEncoder(), [1])], remainder = 'passthrough')
x = ct.fit_transform(x)
# transform the third feature to 67-dim one-hot vector
ct = ColumnTransformer([("Service", OneHotEncoder(), [4])], remainder = 'passthrough')
x = ct.fit_transform(x)
# transform the fourth feature to 11-dim one-hot vector
ct = ColumnTransformer([("Flag", OneHotEncoder(), [71])], remainder = 'passthrough')
x = ct.fit_transform(x)

Labels are also converted to numeric values

In [0]:
# Encode label (5 labels)
y = np.concatenate((y_train, y_test), axis = 0)
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

Data is normalized to ﬂoating point values between 0 and 1

In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

# split to training and testing data
x_train = x[:494021,:]
x_test = x[494021:,:]

y_train = y[:494021]
y_test = y[494021:]


#Construct classifier
The model uses two NDAEs arranged in a stack, is combined with the RF algorithm.

![Stacked NDAE Classification Model](https://drive.google.com/uc?id=1paKfuUYEVtBmOfkrMdzzfElMwGyMd2eC)








In [7]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K
import tensorflow as tf
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.2


Using TensorFlow backend.


In [0]:
class DenseTranspose(keras.layers.Layer):
  def __init__(self, dense, activation=None, **kwargs):
    self.dense = dense
    self.activation = keras.activations.get(activation)
    super().__init__(**kwargs)
  def build(self, batch_input_shape):
    self.biases = self.add_weight(name="bias", initializer="zeros",shape=[self.dense.input_shape[-1]])
    self.W = tf.transpose(self.dense.weights[0]) 
    super().build(batch_input_shape)
  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.dense.input_shape[-1])
  def call(self, inputs):
    z = tf.matmul(inputs, self.W)
    return self.activation(z + self.biases)

Contruct the first autoencoder, and training it.


In [9]:
K.clear_session()
num_hidden = (119, 14, 28, 28)

Dense_11 = Dense(units=num_hidden[1], activation='sigmoid')
Dense_12 = Dense(units=num_hidden[2], activation='sigmoid')
Dense_13 = Dense(units=num_hidden[3], activation='sigmoid')

inputs_1 = Input(shape=(num_hidden[0],))

#Encoder
encoded_11 = Dense_11(inputs_1)
encoded_12 = Dense_12(encoded_11)
encoded_13 = Dense_13(encoded_12)

#Decoder
decoded_11 = DenseTranspose(Dense_13, activation='sigmoid')(encoded_13)
decoded_12 = DenseTranspose(Dense_12, activation='sigmoid')(decoded_11)
outputs_1 = DenseTranspose(Dense_11, activation='sigmoid')(decoded_12)

AE_1=Model(inputs_1, outputs_1)
Encoder_1=Model(inputs_1, decoded_12)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
AE_1.compile(optimizer='rmsprop', loss= 'mse')
AE_1.fit(x_train,x_train,epochs=100,batch_size=256,shuffle=True)


Epoch 1/100
494021/494021 [==============================] - 7s 14us/step - loss: 0.0339
Epoch 2/100
494021/494021 [==============================] - 7s 13us/step - loss: 0.0048
Epoch 3/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0024
Epoch 4/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0020
Epoch 5/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0017
Epoch 6/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0015
Epoch 7/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0014
Epoch 8/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0013
Epoch 9/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0013
Epoch 10/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0012
Epoch 11/100
494021/494021 [==============================] - 6s 13us/step - loss: 0.0012
Epoch 12/100
49402

Getting the input of the last layer to fed to the second autoencoder.

In [0]:
AE_1_encoded_train = Encoder_1.predict(x_train)
AE_1_encoded_test = Encoder_1.predict(x_test)

Contruct the second autoencoder, and training it.

In [0]:
K.clear_session()
num_hidden = (14, 28, 28)

Dense_21 = Dense(units=num_hidden[1], activation='sigmoid')
Dense_22 = Dense(units=num_hidden[2], activation='sigmoid')


inputs_2 = Input(shape=(num_hidden[0],))

#Encoder
encoded_21 = Dense_21(inputs_2)
encoded_22 = Dense_22(encoded_21)


#Decoder
decoded_21 = DenseTranspose(Dense_22, activation='sigmoid')(encoded_22)
outputs_2 = DenseTranspose(Dense_21, activation='sigmoid')(decoded_21)

AE_2=Model(inputs_2, outputs_2)
Encoder_2=Model(inputs_2, decoded_21)

In [13]:
AE_2.compile(optimizer='rmsprop', loss= 'mse')
AE_2.fit(AE_1_encoded_train,AE_1_encoded_train,epochs=100,batch_size=256,shuffle=True)

Epoch 1/100
494021/494021 [==============================] - 5s 10us/step - loss: 0.0268
Epoch 2/100
494021/494021 [==============================] - 5s 10us/step - loss: 0.0012
Epoch 3/100
494021/494021 [==============================] - 5s 10us/step - loss: 6.5561e-04
Epoch 4/100
494021/494021 [==============================] - 5s 10us/step - loss: 2.7392e-04
Epoch 5/100
494021/494021 [==============================] - 5s 10us/step - loss: 1.6577e-04
Epoch 6/100
494021/494021 [==============================] - 5s 10us/step - loss: 1.2919e-04
Epoch 7/100
494021/494021 [==============================] - 5s 10us/step - loss: 1.0199e-04
Epoch 8/100
494021/494021 [==============================] - 5s 10us/step - loss: 7.5854e-05
Epoch 9/100
494021/494021 [==============================] - 5s 10us/step - loss: 5.4787e-05
Epoch 10/100
494021/494021 [==============================] - 5s 10us/step - loss: 4.3360e-05
Epoch 11/100
494021/494021 [==============================] - 5s 11us/step - 

Getting the input of the last layer to fed to RF algorithm.

In [0]:
AE_2_encoded_train = Encoder_2.predict(AE_1_encoded_train)
AE_2_encoded_test = Encoder_2.predict(AE_1_encoded_test)

In [15]:
from sklearn.ensemble import RandomForestClassifier
rfc1 = RandomForestClassifier(n_jobs=-1, n_estimators=10)
rfc1.fit(AE_2_encoded_train, y_train)
rfc1.score(AE_2_encoded_test, y_test)

0.9784775915155662